## MIR PROJECT


### TITLE:

Market Basket Analysis for AHG.


### 1) THE DATASET

The dataset used for this project was retrived from AdventureWorks 2017. 


###### Some of the Python Libraries


In [1]:
##work horses of tables (dataframe) and arrays
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pyodbc
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import mlxtend as ml
from mlxtend.preprocessing import TransactionEncoder

cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=DESKTOP-2NRFU9A;"
                      "Database=AdventureWorks2017;"
                      "Trusted_Connection=yes;")

##### Reading the dataset for Central

In [2]:
#Importing southwest Data from SQL server
# the head is printing the first 5 rows of the data set.
df = pd.read_sql_query(""" 
select PP.Name as Productname
,C.Name as Region
,A.[OrderQty] as OrderQty
,B.[OrderDate] as Date
,CASE WHEN B.[OnlineOrderFlag] = 1 THEN 'Online' ELSE 'Reseller' END AS Sales_Channel
from Production.Product PP
left join Sales.SalesOrderDetail A
LEFT JOIN [Sales].[SalesOrderHeader]  B
ON A.SalesOrderID = B.SalesOrderID
on A.ProductID = PP.ProductID 
LEFT JOIN Sales.SalesTerritory C
ON C.TerritoryID = B.TerritoryID""", cnxn)

df.head()

,Productname,Region,OrderQty,Date,Sales_Channel
0,"Mountain-100 Black, 42",Southeast,1.0,2011-05-31,Reseller
1,"Mountain-100 Black, 44",Southeast,3.0,2011-05-31,Reseller
2,"Mountain-100 Black, 48",Southeast,1.0,2011-05-31,Reseller
3,"Mountain-100 Silver, 38",Southeast,1.0,2011-05-31,Reseller
4,"Mountain-100 Silver, 42",Southeast,1.0,2011-05-31,Reseller


In [3]:
# the tail is printing the last 5 rows of the data set.
df.tail()

,Productname,Region,OrderQty,Date,Sales_Channel
121550,"HL Road Frame - Red, 56",None,NaN,NaT,Reseller
121551,Headlights - Dual-Beam,None,NaN,NaT,Reseller
121552,Flat Washer 7,None,NaN,NaT,Reseller
121553,Thin-Jam Lock Nut 8,None,NaN,NaT,Reseller
121554,Paint - Black,None,NaN,NaT,Reseller


In [4]:
#This is to identify how many columns and roles
print ('Data Dimension for MIR Project data:{}' .format(df.shape))

Data Dimension for MIR Project data:(121555, 5)


In [5]:
#Identifying the various data types
df.dtypes

Productname              object
Region                   object
OrderQty                float64
Date             datetime64[ns]
Sales_Channel            object
dtype: object

In [6]:
#Checking for missing values
df.isnull().sum()

Productname        0
Region           238
OrderQty         238
Date             238
Sales_Channel      0
dtype: int64

There is a little cleanup, we need to do. First, some of the productNames have spaces that need to be removed. 

In [7]:
df['Productname'] = df['Productname'].str.strip()

After the cleanup, we need to consolidate the items into 1 transaction per row with each product. For the sake of keeping the data set small, I’m only looking at Sales_Channels for Reseller. However, in additional code below, I will compare these results to Sales_Channels from Online.

Here’s what the first few columns look like (note, I added some numbers to the columns to illustrate the concept - the actual data in this example is all 0’s):

## Reseller:

In [8]:
basket = (df[df['Sales_Channel']=="Reseller"]
          .groupby(['Date','Productname'])['OrderQty']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Date'))

basket.head()

Productname,AWC Logo Cap,"HL Mountain Frame - Black, 38","HL Mountain Frame - Black, 42","HL Mountain Frame - Black, 48","HL Mountain Frame - Silver, 38","HL Mountain Frame - Silver, 46","HL Mountain Frame - Silver, 48","LL Road Frame - Black, 52","LL Road Frame - Black, 58","LL Road Frame - Red, 44",...,"Touring-3000 Yellow, 50","Touring-3000 Yellow, 54","Touring-3000 Yellow, 58","Touring-3000 Yellow, 62",Water Bottle - 30 oz.,"Women's Mountain Shorts, L","Women's Mountain Shorts, M","Women's Mountain Shorts, S","Classic Vest, L","LL Touring Frame - Blue, 58"
Date,,,,,,,,,,,,,,,,,,,,,
2011-05-31,40.0,4.0,1.0,1.0,2.0,3.0,2.0,19.0,8.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-07-01,103.0,14.0,13.0,19.0,20.0,15.0,14.0,39.0,20.0,38.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-08-01,83.0,17.0,21.0,16.0,13.0,18.0,17.0,24.0,7.0,29.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-08-31,54.0,0.0,7.0,2.0,8.0,3.0,7.0,20.0,16.0,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-10-01,136.0,12.0,22.0,17.0,24.0,17.0,22.0,43.0,37.0,41.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


There are a lot of zeros in the data but we also need to make sure any positive values are converted to a 1 and anything less the 0 is set to 0. This step will complete the one hot encoding of the data:

In [9]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
#basket_sets.drop('POSTAGE', inplace=True, axis=1)

basket_sets.head()

Productname,AWC Logo Cap,"HL Mountain Frame - Black, 38","HL Mountain Frame - Black, 42","HL Mountain Frame - Black, 48","HL Mountain Frame - Silver, 38","HL Mountain Frame - Silver, 46","HL Mountain Frame - Silver, 48","LL Road Frame - Black, 52","LL Road Frame - Black, 58","LL Road Frame - Red, 44",...,"Touring-3000 Yellow, 50","Touring-3000 Yellow, 54","Touring-3000 Yellow, 58","Touring-3000 Yellow, 62",Water Bottle - 30 oz.,"Women's Mountain Shorts, L","Women's Mountain Shorts, M","Women's Mountain Shorts, S","Classic Vest, L","LL Touring Frame - Blue, 58"
Date,,,,,,,,,,,,,,,,,,,,,
2011-05-31,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2011-07-01,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2011-08-01,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2011-08-31,1,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2011-10-01,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


Now that the data is structured properly, we can generate frequent item sets that have a support of at least 70% (this number was chosen so that I could get enough useful examples):

In [10]:
frequent_itemsets = apriori(basket_sets, min_support=0.7, use_colnames=True)
frequent_itemsets.head()

,support,itemsets
0,0.925,(AWC Logo Cap)
1,0.850,"(HL Mountain Frame - Black, 38)"
2,0.900,"(HL Mountain Frame - Black, 42)"
3,0.900,"(HL Mountain Frame - Silver, 38)"
4,0.900,"(HL Mountain Frame - Silver, 46)"


The final step is to generate the rules with their corresponding support, confidence and lift:

In [11]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(HL Mountain Frame - Black, 38)",(AWC Logo Cap),0.850,0.925,0.85,1.000000,1.081081,0.06375,inf
1,(AWC Logo Cap),"(HL Mountain Frame - Black, 38)",0.925,0.850,0.85,0.918919,1.081081,0.06375,1.85
2,"(HL Mountain Frame - Black, 42)",(AWC Logo Cap),0.900,0.925,0.90,1.000000,1.081081,0.06750,inf
3,(AWC Logo Cap),"(HL Mountain Frame - Black, 42)",0.925,0.900,0.90,0.972973,1.081081,0.06750,3.70
4,"(HL Mountain Frame - Silver, 38)",(AWC Logo Cap),0.900,0.925,0.90,1.000000,1.081081,0.06750,inf


That’s all there is to it! Build the frequent items using apriori then build the rules with association_rules .

We can filter the dataframe using standard pandas code. In this case, look for a large lift (1.1) and high confidence (1):

In [24]:
rules[ (rules['lift'] >= 1.1) &
       (rules['confidence'] >= 1) ].head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
30,"(HL Mountain Frame - Black, 38)","(HL Mountain Frame - Black, 42)",0.85,0.9,0.85,1.0,1.111111,0.085,inf
32,"(HL Mountain Frame - Black, 38)","(HL Mountain Frame - Silver, 38)",0.85,0.9,0.85,1.0,1.111111,0.085,inf
34,"(HL Mountain Frame - Black, 38)","(HL Mountain Frame - Silver, 46)",0.85,0.9,0.85,1.0,1.111111,0.085,inf
36,"(HL Mountain Frame - Black, 38)","(LL Road Frame - Black, 52)",0.85,0.9,0.85,1.0,1.111111,0.085,inf
38,"(HL Mountain Frame - Black, 38)","(LL Road Frame - Black, 58)",0.85,0.9,0.85,1.0,1.111111,0.085,inf


In looking at the rules, it seems that the Black and Silver HL Mountain Frame are purchased together and the LL Road Frame, Sport-100 Helmet Red and Black, and Long-Sleeve Logo Jersey, XL  are purchased together with HL Mountain Frame-Black,38 in a manner that is higher than the overall probability would suggest.

At this point, you may want to look at how much opportunity there is to use the popularity of one product to drive sales of another. For instance, we can see that we sell 1,181 HL Mountain Frame - Black, 42 but only 3659 HL Mountain Frame - Black, 38 so maybe we can drive more HL Mountain Frame - Black, 42 sales through recommendations.

In [13]:
basket["HL Mountain Frame - Black, 42"].sum()

1181.0

In [14]:
basket["HL Mountain Frame - Black, 38"].sum()

659.0

## Online:

What is also interesting is to see how the combinations vary by Sales_Channels of purchase. Let’s check out what some popular combinations might be for Online:

In [15]:
basket2 = (df[df['Sales_Channel']=="Online"]
          .groupby(['Date','Productname'])['OrderQty']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Date'))

basket2.head()

Productname,AWC Logo Cap,All-Purpose Bike Stand,Bike Wash - Dissolver,"Classic Vest, L","Classic Vest, M","Classic Vest, S",Fender Set - Mountain,HL Mountain Tire,HL Road Tire,"Half-Finger Gloves, L",...,"Touring-3000 Blue, 62","Touring-3000 Yellow, 44","Touring-3000 Yellow, 50","Touring-3000 Yellow, 54","Touring-3000 Yellow, 58","Touring-3000 Yellow, 62",Water Bottle - 30 oz.,"Women's Mountain Shorts, L","Women's Mountain Shorts, M","Women's Mountain Shorts, S"
Date,,,,,,,,,,,,,,,,,,,,,
2011-05-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-06-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-06-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-06-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-06-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
basket2.tail()

Productname,AWC Logo Cap,All-Purpose Bike Stand,Bike Wash - Dissolver,"Classic Vest, L","Classic Vest, M","Classic Vest, S",Fender Set - Mountain,HL Mountain Tire,HL Road Tire,"Half-Finger Gloves, L",...,"Touring-3000 Blue, 62","Touring-3000 Yellow, 44","Touring-3000 Yellow, 50","Touring-3000 Yellow, 54","Touring-3000 Yellow, 58","Touring-3000 Yellow, 62",Water Bottle - 30 oz.,"Women's Mountain Shorts, L","Women's Mountain Shorts, M","Women's Mountain Shorts, S"
Date,,,,,,,,,,,,,,,,,,,,,
2014-06-26,3.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,1.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1.0,1.0
2014-06-27,2.0,0.0,0.0,0.0,0.0,0.0,5.0,2.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,1.0,0.0
2014-06-28,3.0,0.0,2.0,0.0,0.0,0.0,3.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,2.0,2.0
2014-06-29,2.0,0.0,1.0,0.0,2.0,1.0,1.0,1.0,3.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0
2014-06-30,6.0,1.0,7.0,2.0,0.0,1.0,6.0,3.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,1.0


In [17]:
basket_sets2 = basket2.applymap(encode_units)
basket_sets2.head()

Productname,AWC Logo Cap,All-Purpose Bike Stand,Bike Wash - Dissolver,"Classic Vest, L","Classic Vest, M","Classic Vest, S",Fender Set - Mountain,HL Mountain Tire,HL Road Tire,"Half-Finger Gloves, L",...,"Touring-3000 Blue, 62","Touring-3000 Yellow, 44","Touring-3000 Yellow, 50","Touring-3000 Yellow, 54","Touring-3000 Yellow, 58","Touring-3000 Yellow, 62",Water Bottle - 30 oz.,"Women's Mountain Shorts, L","Women's Mountain Shorts, M","Women's Mountain Shorts, S"
Date,,,,,,,,,,,,,,,,,,,,,
2011-05-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2011-06-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2011-06-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2011-06-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2011-06-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
basket_sets2.tail()

Productname,AWC Logo Cap,All-Purpose Bike Stand,Bike Wash - Dissolver,"Classic Vest, L","Classic Vest, M","Classic Vest, S",Fender Set - Mountain,HL Mountain Tire,HL Road Tire,"Half-Finger Gloves, L",...,"Touring-3000 Blue, 62","Touring-3000 Yellow, 44","Touring-3000 Yellow, 50","Touring-3000 Yellow, 54","Touring-3000 Yellow, 58","Touring-3000 Yellow, 62",Water Bottle - 30 oz.,"Women's Mountain Shorts, L","Women's Mountain Shorts, M","Women's Mountain Shorts, S"
Date,,,,,,,,,,,,,,,,,,,,,
2014-06-26,1,0,0,0,1,0,1,0,1,1,...,0,0,0,0,0,0,1,1,1,1
2014-06-27,1,0,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,1,0,1,0
2014-06-28,1,0,1,0,0,0,1,0,1,1,...,0,0,0,0,0,0,1,0,1,1
2014-06-29,1,0,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,1,1,0,0
2014-06-30,1,1,1,1,0,1,1,1,1,1,...,0,0,0,0,0,0,1,1,0,1


On like Reseller we will 30% min_support for Online Channels:

In [19]:
frequent_itemsets2 = apriori(basket_sets2, min_support=0.30, use_colnames=True)
frequent_itemsets2.head()

,support,itemsets
0,0.348754,(AWC Logo Cap)
1,0.309609,(Bike Wash - Dissolver)
2,0.346085,(Fender Set - Mountain)
3,0.332740,(HL Mountain Tire)
4,0.305160,(HL Road Tire)


In [20]:
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1)
rules2.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(AWC Logo Cap),(Bike Wash - Dissolver),0.348754,0.309609,0.306050,0.877551,2.834389,0.198072,5.638197
1,(Bike Wash - Dissolver),(AWC Logo Cap),0.309609,0.348754,0.306050,0.988506,2.834389,0.198072,56.658363
2,(Fender Set - Mountain),(AWC Logo Cap),0.346085,0.348754,0.343416,0.992288,2.845234,0.222718,84.444840
3,(AWC Logo Cap),(Fender Set - Mountain),0.348754,0.346085,0.343416,0.984694,2.845234,0.222718,42.722420
4,(HL Mountain Tire),(AWC Logo Cap),0.332740,0.348754,0.329181,0.989305,2.836680,0.213137,60.891459


In this case, i will use large lift (2.99) and high confidence (1):

In [21]:
rules2[ (rules2['lift'] >= 2.99) &
        (rules2['confidence'] >= 1)].head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
57324,"(Touring Tire Tube, ML Mountain Tire)","(Mountain Tire Tube, Road Tire Tube, Patch Kit...",0.306050,0.33363,0.306050,1.0,2.997333,0.203942,inf
103008,"(Touring Tire Tube, AWC Logo Cap, ML Mountain ...","(Mountain Tire Tube, Road Tire Tube, Patch Kit...",0.304270,0.33363,0.304270,1.0,2.997333,0.202757,inf
114414,"(AWC Logo Cap, Touring Tire Tube, Sport-100 He...","(Mountain Tire Tube, Road Tire Tube, Patch Kit...",0.318505,0.33363,0.318505,1.0,2.997333,0.212242,inf
130787,"(Fender Set - Mountain, Touring Tire Tube, ML ...","(Mountain Tire Tube, Road Tire Tube, Patch Kit...",0.302491,0.33363,0.302491,1.0,2.997333,0.201571,inf
158560,"(Touring Tire Tube, Mountain Bottle Cage, ML M...","(Mountain Tire Tube, Road Tire Tube, Patch Kit...",0.304270,0.33363,0.304270,1.0,2.997333,0.202757,inf


In [22]:
basket2['Road Tire Tube'].sum()

2376.0

In [23]:
basket2['AWC Logo Cap'].sum()

2190.0